# Basenji-style data preprocessing

### Convert CAGE and DNase bams to bigWigs

In [1]:
! python utils/bam_cov.py ../data/encode/cage/k562/ENCFF366MWI.bam ../data/encode/cage/k562/ENCFF366MWI.bw
! python utils/bam_cov.py ../data/encode/cage/k562/ENCFF754FAU.bam ../data/encode/cage/k562/ENCFF754FAU.bw
! python utils/bam_cov.py ../data/encode/dnase/k562/ENCFF257HEE.bam ../data/encode/dnase/k562/ENCFF257HEE.bw

/pollard/home/sandyfloren/hierarchical/bin/utils/bam_cov.py:35: DeprecationWarning: Please import `gaussian_filter1d` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_filter1d
Reading alignments from BAM. Done in 316s.
Constructing multi-read CSR matrix. Done in 0s.


### Compare CAGE replicates 1 and 2

### Create 6 Mb sequences

In [5]:
import pysam
# Copied from basenji/basenji/genome.py
def load_chromosomes(genome_file):
  """ Load genome segments from either a FASTA file or
          chromosome length table. """

  # is genome_file FASTA or (chrom,start,end) table?
  file_fasta = (open(genome_file).readline()[0] == '>')

  chrom_segments = {}

  if file_fasta:
    fasta_open = pysam.Fastafile(genome_file)
    for i in range(len(fasta_open.references)):
      chrom_segments[fasta_open.references[i]] = [(0, fasta_open.lengths[i])]
    fasta_open.close()

  else:
    for line in open(genome_file):
      a = line.split()
      chrom_segments[a[0]] = [(0, int(a[1]))]

  return chrom_segments

chromosomes = ['chr' + str(i) for i in range(1, 23)] + ['chrX', 'chrY']
chrom_segments = load_chromosomes("/pollard/data/vertebrate_genomes/human/hg38/hg38/hg38.fa")
chrom_segments = {k: v for k, v in chrom_segments.items() if k in chromosomes}

In [9]:
chromosomes = ['chr' + str(i) for i in range(1, 23)] + ['chrX', 'chrY']
num_bins_dict = {k: 0 for k in chromosomes}
with open('../data/graphreg_data/seqs/genome_bins.bed', 'r') as infile:
    for bin in infile:
        chrom, _, _ = bin.split()
        num_bins_dict[chrom] += 1
num_bins_dict

{'chr1': 49791,
 'chr2': 48438,
 'chr3': 39659,
 'chr4': 38042,
 'chr5': 36307,
 'chr6': 34161,
 'chr7': 31869,
 'chr8': 29027,
 'chr9': 27678,
 'chr10': 26759,
 'chr11': 27017,
 'chr12': 26655,
 'chr13': 22872,
 'chr14': 21408,
 'chr15': 20398,
 'chr16': 18067,
 'chr17': 16651,
 'chr18': 16074,
 'chr19': 11723,
 'chr20': 12888,
 'chr21': 9341,
 'chr22': 10163,
 'chrX': 31208,
 'chrY': 11445}

In [8]:
# Write files of 6Mb sequence coordinates
# We do need to use basenji_data_read because of the conflicting pool_width btw. CAGE and DNase, etc.
# As of now we are throwing away the end of the chromosomes instead of including it and padding it to 6 Mb
# this creates 1463 windows

window_size = 6000000
stride = 2000000

with open('../data/graphreg_data/seqs/windows_6Mb_no_gaps.bed', 'w') as file:
    for chrom, segments in chrom_segments.items():
        for segment in segments:
            begin, end = segment
            while begin + window_size <= end:
                file.write(f'{chrom}\t{begin}\t{begin + window_size}\n')
                begin += stride

In [7]:
# Write 5kb bins file
binsize = 5000

with open('../data/graphreg_data/seqs/genome_bins.bed', 'w') as file:
    for chrom, segments in chrom_segments.items():
        for segment in segments:
            begin, end = segment
            while begin + binsize <= end:
                file.write(f'{chrom}\t{begin}\t{begin + binsize}\n')
                begin += binsize

### Save bigWigs as h5

In [10]:
! python utils/basenji_data_read.py -b ../data/encode/exclusion_list/exclusion_list_with_repeats.bed -u sum -w 5000 ../data/encode/cage/k562/ENCFF366MWI.bw ../data/graphreg_data/seqs/windows_6Mb_no_gaps.bed ../data/graphreg_data/1d/k562/cage_rep1.h5
! python utils/basenji_data_read.py -b ../data/encode/exclusion_list/exclusion_list_with_repeats.bed -u sum -w 5000 ../data/encode/cage/k562/ENCFF754FAU.bw ../data/graphreg_data/seqs/windows_6Mb_no_gaps.bed ../data/graphreg_data/1d/k562/cage_rep2.h5
! python utils/basenji_data_read.py -b ../data/encode/exclusion_list/exclusion_list_with_repeats.bed -u sum -w 100 ../data/encode/dnase/k562/ENCFF257HEE.bw ../data/graphreg_data/seqs/windows_6Mb_no_gaps.bed ../data/graphreg_data/1d/k562/dnase.h5
! python utils/basenji_data_read.py -b ../data/encode/exclusion_list/exclusion_list_with_repeats.bed -u sum -w 100 ../data/encode/h3k4me3/k562/ENCFF253TOF.bigWig ../data/graphreg_data/seqs/windows_6Mb_no_gaps.bed ../data/graphreg_data/1d/k562/h3k4me3.h5
! python utils/basenji_data_read.py -b ../data/encode/exclusion_list/exclusion_list_with_repeats.bed -u sum -w 100 ../data/encode/h3k27ac/k562/ENCFF381NDD.bigWig ../data/graphreg_data/seqs/windows_6Mb_no_gaps.bed ../data/graphreg_data/1d/k562/h3k27ac.h5

Targets sum: 36021629.000
^C
Traceback (most recent call last):
  File "/pollard/home/sandyfloren/hierarchical/bin/utils/basenji_data_read.py", line 332, in <module>
    main()
  File "/pollard/home/sandyfloren/hierarchical/bin/utils/basenji_data_read.py", line 116, in main
    seq_cov_nt = genome_cov_open.read(mseq.chr, mseq.start, mseq.end)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pollard/home/sandyfloren/hierarchical/bin/utils/basenji_data_read.py", line 308, in read
    cov = self.cov_open.values(chrm, start, end, numpy=True).astype('float16')
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


In [52]:
import h5py

cage_rep1 = h5py.File('../data/graphreg_data/1d/k562/cage_rep1.h5', 'r')
max(cage_rep1['targets'][6])

5748.0

### Get 3d data

In [ ]:
[]

### Process 3d data with HiCDCPlus

In [53]:
# mamba activate R
# 

### Extract graphs from 3d data 

### Save 3D data as npz

### Save seqs, 1d and 3d data as TFRecords

# Reproduce Seq-GraphReg results from Karbalayghareh et al.

### K562 cells

In [ ]:
! python Seq-CNN_base.py -c K562 -p $data_path -v 3,13 -t 4,14

# Benchmark Seq-GraphReg with 1 Mb window size